# GLS Analysis of PGR Peak Mean Data

This notebook performs Generalized Least Squares (GLS) analysis on PGR peak mean data with two factors:
- **Uterus-specific**: Presence/absence of uterus-specific effects
- **Sensory-specific**: Presence/absence of sensory-specific effects

The analysis uses unequal variance modeling to account for heteroscedasticity across groups.

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

print("Libraries loaded successfully")

Libraries loaded successfully


## Data Loading and Preprocessing

The data consists of 4 groups based on the combination of two binary factors:
- Group 1: Uterus-specific=0, Sensory-specific=0
- Group 2: Uterus-specific=1, Sensory-specific=0  
- Group 3: Uterus-specific=0, Sensory-specific=1
- Group 4: Uterus-specific=1, Sensory-specific=1

In [2]:
def read_input(file_name):
    """
    Read CSV file with 4 rows representing different experimental groups
    """
    with open(file_name, 'r') as f:
        lines = f.readlines()
    
    if len(lines) != 4:
        raise ValueError("CSV file must contain exactly 4 rows, one for each group.")
    
    # Group mapping: (uterus_specific, sensory_specific)
    group_info = [(0, 0), (1, 0), (0, 1), (1, 1)]
    
    data_list = []
    for i, line in enumerate(lines):
        values = [float(x.strip()) for x in line.strip().split(',')]
        uterus_val, sensory_val = group_info[i]
        
        for value in values:
            data_list.append({
                'uterus_specific': uterus_val,
                'sensory_specific': sensory_val,
                'peak_mean': value
            })
    
    df = pd.DataFrame(data_list)
    df['uterus_specific'] = df['uterus_specific'].astype('category')
    df['sensory_specific'] = df['sensory_specific'].astype('category')
    df['group'] = df['uterus_specific'].astype(str) + '_' + df['sensory_specific'].astype(str)
    
    return df

# Load the data
print("Loading data from pgr_peak_mean.csv...")
df_pressure = read_input('pgr_peak_mean.csv')
print(f"Data loaded successfully!")
print(f"Total observations: {df_pressure.shape[0]}")
print(f"Variables: {list(df_pressure.columns)}")

Loading data from pgr_peak_mean.csv...
Data loaded successfully!
Total observations: 29
Variables: ['uterus_specific', 'sensory_specific', 'peak_mean', 'group']


In [3]:
# Display data summary
print("\n=== DATA SUMMARY ===")
print("\nGroup sample sizes:")
group_counts = df_pressure.groupby('group').size()
for group, count in group_counts.items():
    uterus, sensory = group.split('_')
    print(f"  Uterus-specific={uterus}, Sensory-specific={sensory}: {count} observations")

print("\nDescriptive statistics by group:")
group_statistics = df_pressure.groupby('group')['peak_mean'].agg(['count', 'mean', 'std', 'var']).round(4)
for group, group_stat in group_statistics.iterrows():
    uterus, sensory = group.split('_')
    print(f"  Uterus-specific={uterus}, Sensory-specific={sensory}:")
    print(f"    Mean: {group_stat['mean']:.4f}, SD: {group_stat['std']:.4f}, Variance: {group_stat['var']:.4f}")

print("\nFirst few rows of data:")
display(df_pressure.head(10))


=== DATA SUMMARY ===

Group sample sizes:
  Uterus-specific=0, Sensory-specific=0: 8 observations
  Uterus-specific=0, Sensory-specific=1: 6 observations
  Uterus-specific=1, Sensory-specific=0: 7 observations
  Uterus-specific=1, Sensory-specific=1: 8 observations

Descriptive statistics by group:
  Uterus-specific=0, Sensory-specific=0:
    Mean: 135.4096, SD: 54.6974, Variance: 2991.8106
  Uterus-specific=0, Sensory-specific=1:
    Mean: 82.3763, SD: 10.2915, Variance: 105.9148
  Uterus-specific=1, Sensory-specific=0:
    Mean: 108.3287, SD: 50.1021, Variance: 2510.2186
  Uterus-specific=1, Sensory-specific=1:
    Mean: 67.8432, SD: 25.2965, Variance: 639.9113

First few rows of data:


,uterus_specific,sensory_specific,peak_mean,group
0,0,0,76.995995,0_0
1,0,0,163.805111,0_0
2,0,0,140.359678,0_0
3,0,0,63.597187,0_0
4,0,0,151.416587,0_0
5,0,0,204.333643,0_0
6,0,0,84.531180,0_0
7,0,0,198.237405,0_0
8,1,0,92.438752,1_0
9,1,0,65.577130,1_0


## GLS Analysis with Unequal Variance

We perform a GLS analysis that accounts for unequal variances across groups. This approach:
1. Calculates group-specific variances
2. Uses inverse variance weighting in the GLS model
3. Tests for main effects of both factors using one-sided tests

In [4]:
def one_sided_test_gls(model_result, coef_name):
    """
    Perform one-sided t-test for a coefficient in GLS model
    """
    coef_val = model_result.params[coef_name]
    t_val = model_result.tvalues[coef_name]
    df_res = model_result.df_resid
    
    if coef_val < 0:
        p_val = stats.t.cdf(t_val, df=df_res)
    else:
        p_val = 1 - stats.t.cdf(t_val, df=df_res)
    
    return {'p_value': p_val, 'beta': coef_val, 't_value': t_val}

print("\n=== GLS ANALYSIS WITH UNEQUAL VARIANCE ===")

# Calculate group variances
group_vars = df_pressure.groupby('group')['peak_mean'].var()
print("\nGroup variances:")
for group, var_val in group_vars.items():
    uterus, sensory = group.split('_')
    print(f"  Uterus-specific={uterus}, Sensory-specific={sensory}: {var_val:.4f}")

# Create weights (inverse variance)
weights_map = {}
for group_name, var_val in group_vars.items():
    weights_map[group_name] = 1.0 / var_val

df_pressure['weight_vals'] = df_pressure['group'].apply(lambda x: weights_map[x])
weights = df_pressure['weight_vals'].values

print("\nInverse variance weights:")
for group, weight in weights_map.items():
    uterus, sensory = group.split('_')
    print(f"  Uterus-specific={uterus}, Sensory-specific={sensory}: {weight:.6f}")


=== GLS ANALYSIS WITH UNEQUAL VARIANCE ===

Group variances:
  Uterus-specific=0, Sensory-specific=0: 2991.8106
  Uterus-specific=0, Sensory-specific=1: 105.9148
  Uterus-specific=1, Sensory-specific=0: 2510.2186
  Uterus-specific=1, Sensory-specific=1: 639.9113

Inverse variance weights:
  Uterus-specific=0, Sensory-specific=0: 0.000334
  Uterus-specific=0, Sensory-specific=1: 0.009442
  Uterus-specific=1, Sensory-specific=0: 0.000398
  Uterus-specific=1, Sensory-specific=1: 0.001563


In [5]:
# Fit GLS model
X_design = sm.add_constant(pd.get_dummies(df_pressure[['uterus_specific', 'sensory_specific']], drop_first=True))
y = df_pressure['peak_mean'].values

print("\nDesign matrix columns:")
print(list(X_design.columns))

# Fit the GLS model with unequal variance
model_unequal = sm.GLS(y, X_design, sigma=np.diag(1.0/weights)).fit()

print("\n=== MODEL RESULTS ===")
print(f"\nModel summary:")
print(f"  R-squared: {model_unequal.rsquared:.4f}")
print(f"  AIC: {model_unequal.aic:.4f}")
print(f"  BIC: {model_unequal.bic:.4f}")
print(f"  Log-likelihood: {model_unequal.llf:.4f}")
print(f"  Degrees of freedom (residual): {model_unequal.df_resid}")


Design matrix columns:
['const', 'uterus_specific_1', 'sensory_specific_1']

=== MODEL RESULTS ===

Model summary:
  R-squared: 0.3197
  AIC: 282.7058
  BIC: 286.8077
  Log-likelihood: -138.3529
  Degrees of freedom (residual): 26.0


## Statistical Tests for Main Effects

In [6]:
# Perform one-sided tests for main effects
test_uterus = one_sided_test_gls(model_unequal, "uterus_specific_1")
test_sensory = one_sided_test_gls(model_unequal, "sensory_specific_1")

print("\n=== MAIN EFFECTS ANALYSIS ===")
print("\n--- UTERUS-SPECIFIC EFFECT ---")
print(f"  Coefficient (β): {test_uterus['beta']:.6f}")
print(f"  t-value: {test_uterus['t_value']:.6f}")
print(f"  One-sided p-value: {test_uterus['p_value']:.6f}")
if test_uterus['p_value'] < 0.05:
    significance = "SIGNIFICANT"
    direction = "increases" if test_uterus['beta'] > 0 else "decreases"
    print(f"  Result: {significance} - Uterus-specific factor {direction} peak mean")
else:
    print(f"  Result: NOT SIGNIFICANT")

print("\n--- SENSORY-SPECIFIC EFFECT ---")
print(f"  Coefficient (β): {test_sensory['beta']:.6f}")
print(f"  t-value: {test_sensory['t_value']:.6f}")
print(f"  One-sided p-value: {test_sensory['p_value']:.6f}")
if test_sensory['p_value'] < 0.05:
    significance = "SIGNIFICANT"
    direction = "increases" if test_sensory['beta'] > 0 else "decreases"
    print(f"  Result: {significance} - Sensory-specific factor {direction} peak mean")
else:
    print(f"  Result: NOT SIGNIFICANT")

print("\n=== INTERPRETATION ===")
print("\nThe GLS model accounts for unequal variances across experimental groups.")
print("One-sided tests evaluate whether each factor has a directional effect on peak mean values.")
print("\nCoefficient interpretation:")
print(f"  - Uterus-specific coefficient: Expected change in peak mean when uterus-specific factor changes from 0 to 1")
print(f"  - Sensory-specific coefficient: Expected change in peak mean when sensory-specific factor changes from 0 to 1")


=== MAIN EFFECTS ANALYSIS ===

--- UTERUS-SPECIFIC EFFECT ---
  Coefficient (β): -16.008792
  t-value: -1.752212
  One-sided p-value: 0.045763
  Result: SIGNIFICANT - Uterus-specific factor decreases peak mean

--- SENSORY-SPECIFIC EFFECT ---
  Coefficient (β): -47.114303
  t-value: -3.327803
  One-sided p-value: 0.001310
  Result: SIGNIFICANT - Sensory-specific factor decreases peak mean

=== INTERPRETATION ===

The GLS model accounts for unequal variances across experimental groups.
One-sided tests evaluate whether each factor has a directional effect on peak mean values.

Coefficient interpretation:
  - Uterus-specific coefficient: Expected change in peak mean when uterus-specific factor changes from 0 to 1
  - Sensory-specific coefficient: Expected change in peak mean when sensory-specific factor changes from 0 to 1


## Full Model Summary

In [7]:
print("\n=== COMPLETE MODEL SUMMARY ===")
print(model_unequal.summary())


=== COMPLETE MODEL SUMMARY ===
                            GLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.320
Model:                            GLS   Adj. R-squared:                  0.267
Method:                 Least Squares   F-statistic:                     6.108
Date:                Thu, 31 Jul 2025   Prob (F-statistic):            0.00669
Time:                        10:08:05   Log-Likelihood:                -138.35
No. Observations:                  29   AIC:                             282.7
Df Residuals:                      26   BIC:                             286.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
cons